# advanced 00

<a target="_blank" href="https://colab.research.google.com/github/husisy/numqi/blob/main/docs/application/tomography/advanced00.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import numpy as np
import collections

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

## Optimal Pauli UD

In previous notebook, we shows how to certificate the unique determinedness of a Pauli measurement. In this notebook, we will show how to find the optimal Pauli UD.

TODO explain hyperparameter

In [ ]:
seed = 233 #seed=233 can find the optimal UD for 3-qubit (size=31), you can try other seed
num_qubit = 3
num_repeat = {2:10, 3:10, 4:80, 5:80}[num_qubit]
num_init_sample = {2:0, 3:10, 4:80, 5:400}[num_qubit]
matrix_subspace = numqi.gate.get_pauli_group(num_qubit, use_sparse=True)
# index=0 is for the identity matrix
kwargs = {'num_repeat':num_repeat,  'num_init_sample':num_init_sample, 'indexF':[0], 'tag_print':True, 'seed':seed}
ud_pauli_index_list = numqi.unique_determine.find_optimal_UD('udp', num_round=2, mat_list=matrix_subspace, **kwargs)
# ud_pauli_index_list = numqi.unique_determine.load_pauli_ud_example(num_qubit) #load the UD stored in numqi package
print('founded UD optimal')
for ind0,pauli_index in enumerate(ud_pauli_index_list):
    pauli_str = ' '.join([numqi.gate.pauli_index_to_str(x, num_qubit) for x in pauli_index])
    print(f'[round-{ind0}][size={len(pauli_index)}]', pauli_str)

Numerically, we find that, for Pauli measurements, UDP implies UDA. As you can see above, we search for the optimal UDP Pauli measurements. Below, let's verify that the optimal UDP Pauli measurements are also UDA.

In [ ]:
mat_list = [numqi.unique_determine.get_matrix_list_indexing(matrix_subspace, x) for x in ud_pauli_index_list]
is_uda_list = numqi.unique_determine.check_UD('uda', mat_list, num_repeat=num_repeat*5)
for ind0, (pauli_index, (tag, loss)) in enumerate(zip(ud_pauli_index_list, is_uda_list)):
    print(f'[round-{ind0}][size={len(pauli_index)}] UDA={tag}, loss={loss:.5f}')


## Clifford equivalence

Let $A$ be a UDA or UDP measurement scheme, then applying a unitary transform $A'=UAU^\dagger$ remains a UDA or UDP measurement scheme.

Clifford operators are unitary operators that map Pauli operators to Pauli operators. Therefore, we can apply a Clifford operator to a UD Pauli measurement to obtain another UD Pauli measurement.

In [ ]:
num_qubit = 2
size = 11
ud_pauli_index = numqi.unique_determine.load_pauli_ud_example(num_qubit, tag_group_by_size=True)[size][0]
equivalent_list = numqi.gate.get_pauli_subset_equivalent(ud_pauli_index, num_qubit)

ud_pauli_str = ' '.join(numqi.gate.pauli_index_to_str(np.array(ud_pauli_index), num_qubit))
equivalent_str_list = [' '.join(numqi.gate.pauli_index_to_str(np.array(x), num_qubit)) for x in equivalent_list]
print(f'UD: {ud_pauli_str}')
for ind0, x in enumerate(equivalent_str_list):
    print(f'equivalent-{ind0}: {x}')

For 3-qubits optimal UD Pauli measurements with 31 operators, we can find `30240` Clifford equivalent schemes in about 4 minutes.

In [ ]:
## skip in generating documentation website

# num_qubit = 3
# size = 31
# ud_pauli_index = numqi.unique_determine.load_pauli_ud_example(num_qubit, tag_group_by_size=True)[size][0]
# equivalent_list = numqi.gate.get_pauli_subset_equivalent(ud_pauli_index, num_qubit, use_tqdm=True)
# print('len(equivalent_list):', len(equivalent_list))

We can also list all UD Pauli subsets and check their Clifford equivalence relation.

This calculation should only be done for 2-qubits. 3-qubits is already too large.

In [ ]:
all_equivalent_set = numqi.gate.get_pauli_all_subset_equivalent(num_qubit=2)

z0 = {'withI':dict(), 'noI':dict()}
tmp0 = dict()
for key,value in all_equivalent_set.items():
    z0['withI'][key] = [x for x in value if (0 in sorted(x)[0])]
for key,value in all_equivalent_set.items():
    z0['noI'][key] = [x for x in value if (0 not in sorted(x)[0])]

for key,value in z0.items():
    print(key)
    for subset_order,z0 in value.items():
        tmp0 = collections.Counter([len(x) for x in z0])
        tmp0 = sorted(tmp0.items(),key=lambda x:x[0])
        tmp0 = '+'.join([f'{v}x{k}' for k,v in tmp0])
        print(f'#[len={subset_order}]={len(z0)}, {tmp0}')
    print()

## UD in other Hermitian matrix subspace

Our numerical algorithm can also find the optimal in any Hermitian matrix subspace. Usually, UDP is not equivalent to UDA in other Hermitian matrix subspace.

For example, UD Gell-Mann measurements.

In [ ]:
num_qudit = 2
dim_qudit = 3
num_repeat = {(2,3):80, (1,3):10}[(num_qudit,dim_qudit)]
num_init_sample = {(2,3):10, (1,3):0}[(num_qudit,dim_qudit)]

gellmann_basis = numqi.gellmann.all_gellmann_matrix(num_qudit, tensor_n=dim_qudit, with_I=True)
# last item is identity, so put it in indexF (fixed index)
z0 = numqi.unique_determine.find_optimal_UD('udp', num_round=1, mat_list=gellmann_basis,
            indexF=[len(gellmann_basis)-1], num_repeat=num_repeat, num_init_sample=num_init_sample, tag_print=True)
# all_index = numqi.unique_determine.save_index_to_file('gellmann-UD.json', key=f'{num_qudit},{dim_qudit},udp', index=z0)
matB_list = [gellmann_basis[x] for x in z0]
numqi.unique_determine.check_UD('udp', matB_list, num_repeat, early_stop_threshold=1e-4, converge_tol=1e-7, dtype='float64', num_worker=19)


For example, UD in qutrit projector

In [ ]:
## 2 qutrit, projector
matrix_subspace = numqi.unique_determine.get_qutrit_projector_basis(num_qutrit=2)
z0 = numqi.unique_determine.find_optimal_UD('udp', num_round=1, mat_list=matrix_subspace, indexF=[0],
        early_stop_threshold=0.001, num_repeat=80, num_init_sample=150, converge_tol=1e-6, tag_print=True)